# Setup stuff

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import csv

from IPython.display import display, HTML

from treasurer.data import DataTools
from treasurer.summary import SummaryTools

idr_aud_conversion_factor = 10_000
idr_aud_col_equiv = [
    ('IDR - Expenses', 'AUD - Expenses'),
    ('IDR - Income', 'AUD - Income'),
    ('IDR - Account Balance', 'AUD - Balance'),
]

analysis_fy = 2018
analysis_fy_str = 'FY {}-{}'.format(analysis_fy-2000, analysis_fy-1999)

In [2]:
data_dir = '../../data/'
img_dir = '../../images/'
file_bang, file_indo = ('sies-bangalow-20190912-01', 'sies-indonesia-20190912-01')

bang = DataTools.load_data_file(data_dir, file_bang)
bang['Account'] = 'Bangalow'

indo = DataTools.load_data_file(data_dir, file_indo)
indo['Account'] = 'Indonesia'

indo['Non-SIES'] = (~indo['Via The SIES Account'])
indo = indo.drop('Via The SIES Account', axis=1)

indo['Internal'] = indo['Internal Transaction']
indo = indo.drop('Internal Transaction', axis=1)

for a, b in idr_aud_col_equiv:
    indo[b] = indo[a] / idr_aud_conversion_factor
    indo = indo.drop(a, axis=1)
    
full = bang.append(indo, sort=False)
# full = full.sort_values(by='date')

full = full.rename(columns={
    'AUD - Expenses': 'Expenses',
    'AUD - Income': 'Income',
    'AUD - Balance': 'Balance',
})

# print(full.dtypes)
# print(full.shape)
# print(full[(full['Expenses'].isnull() & full['Income'].isnull())])

In [3]:
project_groups = {
    'Expenses': {
        'Administration-Expense': 'Fees',
        'Administrative-Expense': 'Fees',
        'Bank-Transfer-Expense': 'Fees',
        'ALeRT-Tour-Donation': 'Volunteer Program Costs',
        'SCU-Tour-Payments': 'Volunteer Program Costs',
        'General-Tour': 'Volunteer Program Costs',
        'Friends-of-the-National-Park-Foundation': 'Orangutan Conservation Projects',
        'Gunung-Palung-National-Park': 'Orangutan Conservation Projects',
        'Orangutan-Green-Projects': 'Orangutan Conservation Projects',
        'Yayasan-Orangutan-Sumatra-Lestari': 'Orangutan Conservation Projects',
        'Susukan-Baru-Project': 'Habitat Regeneration Projects',
        'Way-Kambas-National-Park-Firefighting-Project': 'Habitat Regeneration Projects',
        'Way-Kambas-National-Park-Reforestation-Project': 'Habitat Regeneration Projects',
        'Tour-Expense-Cash-Withdrawal': 'Eco-Tours Expenses',
        'International-Park-Tours': 'Eco-Tours Expenses',
        'Way-Kambas-National-Park-General': 'Other Projects',
        'Recurring-Sponsorships': 'Other Projects',
        'Way-Kambas-National-Park-Elephant-Project': 'Elephant Conservation Projects',
        'Way-Kambas-National-Park-WWD-Project': 'White Wing Duck Conservation Projects',
        'Payment for Tshirts': 'Volunteer Program Costs',
        'Initial-Balance': 'ERRORS',
        'Transfer-Error': 'ERRORS',
        'General-Donation': 'ERRORS',
        'General-Donation-via-PayPal': 'ERRORS',
        'SCU-Fundraising': 'ERRORS',
        'Volunteer-Tour-Donation': 'ERRORS',
        'Transfer-to-the-Bali-Account': 'ERRORS',
        np.nan: 'ERRORS',
    },
    'Income': {
        'Initial-Balance': 'Balance',
        'Balance': 'Balance',
        'Donation-from-Taronga': 'Grants',
        'Susukan-Baru-Project': 'Grants',
        'General-Donation-via-Planet-Funder': 'Volunteer Program Income',
        'SCU-Fundraising': 'Volunteer Program Income',
        'SCU-Tour-Payments': 'Volunteer Program Income',
        'Orangutan-Green-Projects': 'Orangutan Conservation Campaign',
        'Way-Kambas-National-Park-Elephant-Project': 'Elephant Conservation Campaign',
        'Way-Kambas-National-Park-WWD-Project': 'White Wing Duck Conservation Campaign',
        'General-Tour': 'Eco-Tours Income',
        'International-Park-Tours': 'Eco-Tours Income',
        'Steppes-Travel-Tour': 'Eco-Tours Income',
        'Volunteer-Tour-Donation': 'Eco-Tours Income',
        'Donation-for-Tigers': 'Public Donations',
        'General-Donation': 'Public Donations',
        'General-Donation-via-PayPal': 'Public Donations',
        'Recurring-Sponsorships': 'Public Donations',
        'Way-Kambas-National-Park-Reforestation-Project': 'Habitat Regeneration Campaign',
        'Way-Kambas-National-Park-General': 'Other Projects',
        'Yayasan-Orangutan-Sumatra-Lestari': 'Orangutan Conservation Projects',
        'Administration-Expense': 'ERRORS',
        'Bank-Transfer-Expense': 'ERRORS',
        'Tour-Expense-Cash-Withdrawal': 'ERRORS',
        'Transfer-to-the-Bali-Account': 'ERRORS',
        np.nan: 'ERRORS',
    },
}

sort_order = {
    'Expenses': (
        'Eco-Tours Expenses',
        'Elephant Conservation Projects',
        'Fees',
        'Habitat Regeneration Projects',
        'Volunteer Program Costs',
        'White Wing Duck Conservation Projects',
    ),
    'Income': (
        'Eco-Tours Income',
        'Habitat Regeneration Campaign',
        'Public Donations',
        'Volunteer Program Income',
        'White Wing Duck Conservation Campaign',
    )
}

In [4]:
arrows = {'Expenses': [0.96, 0.5, -0.025, -0.018]}
SummaryTools.test_for_missing_groups(full, project_groups)
summaries = SummaryTools.pretty_summary(full, project_groups, analysis_fy)
for i, s in summaries:
    # s = s.reindex(sort_order[i])
    # SummaryTools.pie_chart(s.to_frame(), i, img_dir, arrows)
    s['Total'] = s.sum()
    s.name = analysis_fy_str
    s = s.to_frame()
    s['%'] = s[analysis_fy_str] / s[analysis_fy_str].sum() * 200
    s = s[['%', analysis_fy_str]]
    formatters = {
        analysis_fy_str: SummaryTools.dollar_formatter,
        '%': lambda x: '{:.1f}%'.format(x),
    }
    display(HTML(s.to_html(formatters=formatters)))
    # print(s.reindex(sort_order[i]))

,%,FY 18-19
Elephant Conservation Projects,7.4%,"$-7,000"
Fees,0.3%,$-280
Habitat Regeneration Projects,84.5%,"$-80,000"
Volunteer Program Costs,2.6%,"$-2,416"
White Wing Duck Conservation Projects,5.3%,"$-5,000"
Total,100.0%,"$-94,696"


,%,FY 18-19
Eco-Tours Income,2.6%,"$2,500"
Habitat Regeneration Campaign,21.8%,"$20,581"
Public Donations,31.4%,"$29,670"
Volunteer Program Income,32.6%,"$30,840"
White Wing Duck Conservation Campaign,11.6%,"$11,000"
Total,100.0%,"$94,592"


In [33]:
account = 'Bangalow'
to_drop = ['Internal', 'Non-SIES',]

def display_fy_summary_for_acc(df, acc):
    s = SummaryTools.fy_summary(full[full['Account'] == acc])
    for col in to_drop:
        s = s.drop(col, axis=1)
    
    print('Summary by Financial Year: {} account only'.format(acc))
    display(HTML(s.to_html(formatters=SummaryTools.make_formatters())))
    
def display_fy_summary(df):
    s = SummaryTools.fy_summary(full)
    for col in to_drop:
        s = s.drop(col, axis=1)
    
    print('Summary by Financial Year for Both Accounts')
    display(HTML(s.to_html(formatters=SummaryTools.make_formatters())))
    return s
    
# for acc in ('Bangalow', 'Indonesia'):
#     display_fy_summary_for_acc(full, acc)
    
summary = display_fy_summary(full)

display(HTML(pd.DataFrame(
    {analysis_fy_str: [
        summary['Income'].iloc[-1],
        summary['Expenses'].iloc[-1],
        summary['Balance'].iloc[-2],
        summary['Balance'].iloc[-1],
    ]},
    index=['Income', 'Expenses', 'SOFY Balance', 'EOFY Balance']
).to_html(formatters={analysis_fy_str: SummaryTools.dollar_formatter})))

Summary by Financial Year for Both Accounts


,Expenses,Income,Balance
date,,,
2012,"$-32,537","$51,596","$19,059"
2013,"$-150,905","$158,719","$26,873"
2014,"$-100,259","$100,382","$26,997"
2015,"$-210,249","$213,660","$30,408"
2016,"$-268,631","$308,310","$70,087"
2017,"$-216,159","$163,083","$19,976"
2018,"$-94,696","$94,592","$19,872"


,FY 18-19
Income,"$94,592"
Expenses,"$-94,696"
SOFY Balance,"$19,976"
EOFY Balance,"$19,872"


In [6]:
account = 'Bangalow'
to_drop = ['Balance', 'Internal', 'Non-SIES',]

print('Summary by Project Code: financial year starting {} and {} account only'.format(analysis_fy, account))
s = SummaryTools.project_summary(full[(
    (full['Account'] == account) &
    SummaryTools.fy_filter(full, analysis_fy)
)])

for col in to_drop:
    s = s.drop(col, axis=1)
    
display(HTML(s.to_html(formatters=SummaryTools.make_formatters())))

Summary by Project Code: financial year starting 2018 and Bangalow account only


,Expenses,Income
Project Codes,,
Administrative-Expense,$-250,$0
Bank-Transfer-Expense,$-30,$0
General-Donation,$0,"$23,010"
General-Donation-via-PayPal,$0,"$6,660"
International-Park-Tours,$0,"$2,500"
Payment for Tshirts,"$-1,208",$0
SCU-Fundraising,$0,"$30,840"
SCU-Tour-Payments,"$-1,208",$0
Way-Kambas-National-Park-Elephant-Project,"$-7,000",$0


In [7]:
to_drop = ['Balance', 'Internal', 'Non-SIES',]

print('Summary by Project Code: financial year starting {}, both accounts'.format(analysis_fy))
s = full[SummaryTools.fy_filter(full, analysis_fy)].groupby('Project Codes').sum()

for col in to_drop:
    s = s.drop(col, axis=1)

s.loc['Total', ['Expenses', 'Income']] = s.sum()
s = s[['Income', 'Expenses']]
s = s.reindex([
    'General-Donation',
    'General-Donation-via-PayPal',
    'SCU-Fundraising',
    'SCU-Tour-Payments',
    'Way-Kambas-National-Park-Elephant-Project',
    'Way-Kambas-National-Park-Reforestation-Project',
    'Way-Kambas-National-Park-WWD-Project',
    'International-Park-Tours',
    'Payment for Tshirts',
    'Administrative-Expense',
    'Bank-Transfer-Expense',
    'Total'
])
    
display(HTML(s.to_html(formatters=SummaryTools.make_formatters())))

Summary by Project Code: financial year starting 2018, both accounts


,Income,Expenses
Project Codes,,
General-Donation,"$23,010",$0
General-Donation-via-PayPal,"$6,660",$0
SCU-Fundraising,"$30,840",$0
SCU-Tour-Payments,$0,"$-1,208"
Way-Kambas-National-Park-Elephant-Project,$0,"$-7,000"
Way-Kambas-National-Park-Reforestation-Project,"$20,581","$-80,000"
Way-Kambas-National-Park-WWD-Project,"$11,000","$-5,000"
International-Park-Tours,"$2,500",$0
Payment for Tshirts,$0,"$-1,208"
